<center><img src='https://drive.google.com/uc?id=1_utx_ZGclmCwNttSe40kYA6VHzNocdET' height="60"></center>

AI TECH - Akademia Innowacyjnych Zastosowań Technologii Cyfrowych. Program Operacyjny Polska Cyfrowa na lata 2014-2020
<hr>

<center><img src='https://drive.google.com/uc?id=1BXZ0u3562N_MqCLcekI-Ens77Kk4LpPm'></center>

<center>
Projekt współfinansowany ze środków Unii Europejskiej w ramach Europejskiego Funduszu Rozwoju Regionalnego
Program Operacyjny Polska Cyfrowa na lata 2014-2020,
Oś Priorytetowa nr 3 "Cyfrowe kompetencje społeczeństwa" Działanie  nr 3.2 "Innowacyjne rozwiązania na rzecz aktywizacji cyfrowej"
Tytuł projektu:  „Akademia Innowacyjnych Zastosowań Technologii Cyfrowych (AI Tech)”
    </center>

# Linear regression

In this exercise you will use linear regression to predict flat prices. Training will be handled via gradient descent and we will:
* have multiple features (i.e. variables used to make the prediction),
* employ some basic feature engineering,
* work with a non-standard loss function.

Let's start with getting the data.

In [16]:
%matplotlib inline

!wget -O mieszkania.csv https://www.dropbox.com/s/zey0gx91pna8irj/mieszkania.csv?dl=1
!wget -O mieszkania_test.csv https://www.dropbox.com/s/dbrj6sbxb4ayqjz/mieszkania_test.csv?dl=1

zsh:1: no matches found: https://www.dropbox.com/s/zey0gx91pna8irj/mieszkania.csv?dl=1
zsh:1: no matches found: https://www.dropbox.com/s/dbrj6sbxb4ayqjz/mieszkania_test.csv?dl=1


In [17]:
!head mieszkania.csv mieszkania_test.csv

==> mieszkania.csv <==
m2,dzielnica,ilość_sypialni,ilość_łazienek,rok_budowy,parking_podziemny,cena
104,mokotowo,2,2,1940,1,780094
43,ochotowo,1,1,1970,1,346912
128,grodziskowo,3,2,1916,1,523466
112,mokotowo,3,2,1920,1,830965
149,mokotowo,3,3,1977,0,1090479
80,ochotowo,2,2,1937,0,599060
58,ochotowo,2,1,1922,0,463639
23,ochotowo,1,1,1929,0,166785
40,mokotowo,1,1,1973,0,318849

==> mieszkania_test.csv <==
m2,dzielnica,ilość_sypialni,ilość_łazienek,rok_budowy,parking_podziemny,cena
71,wolowo,2,2,1912,1,322227
45,mokotowo,1,1,1938,0,295878
38,mokotowo,1,1,1999,1,306530
70,ochotowo,2,2,1980,1,553641
136,mokotowo,3,2,1939,1,985348
128,wolowo,3,2,1983,1,695726
23,grodziskowo,1,1,1975,0,99751
117,mokotowo,3,2,1942,0,891261
65,ochotowo,2,1,2002,1,536499


In [29]:
import numpy as np
import pandas as pd
from typing import Tuple

Each row in the data represents a separate property. Our goal is to use the data from `mieszkania.csv` to create a model that can predict a property's price (i.e. `cena`) given its features (i.e. `m2,dzielnica,ilosc_sypialni,ilosc_lazienek,rok_budowy,parking_podziemny`).

From now on, we should interfere only with `mieszkania.csv` (dubbed the training dataset) to make our decisions and create the model. The (only) purpose of `mieszkania_test.csv` is to test our model on **unseen** data.

Our predictions should minimize the so-called mean squared logarithmic error:
$$
MSLE = \frac{1}{n} \sum_{i=1}^n (\log(1+y_i) - \log(1+p_i))^2,
$$
where $y_i$ is the ground truth, and $p_i$ is our prediction.

Let's start with implementing the loss function.

In [31]:
def msle(ys, ps):
    assert len(ys) == len(ps)
    return np.mean((np.log(1 + ys) - np.log(1 + ps)) ** 2)

The simplest model is predicting the same constant for each instance. Test your implementation of msle against outputing the mean price.

In [32]:
def load(name: str) -> Tuple[np.ndarray, np.array]:
    data = pd.read_csv(name)
    x = data.loc[:, data.columns != 'cena'].to_numpy()
    y = data['cena'].to_numpy()
    return x, y

xs, ys = load('mieszkania.csv')
b, f = xs.shape

In [33]:
###################################################
# TODO: Compute msle for outputing the mean price #
###################################################

ps1 = np.mean(ys) * np.ones(b)
msle_for_mean = msle(ys, ps1)

msle_for_mean

0.3915253538257009

Recall that outputing the mean minimzes $MSE$. However, we're now dealing with $MSLE$.

Think of a constant that should result in the lowest $MSLE$.

In [21]:
#############################################
# TODO: Find this constant and compute msle #
#############################################
magic_constant = (np.exp(np.mean(np.log(1 + ys))) - 1)

magic_loss =  msle(ys, magic_constant  * np.ones(b))
magic_loss

0.3648896122133589

Now, let's implement a standard linear regression model.

In [22]:
def load2(name: str) -> Tuple[np.ndarray, np.array, np.ndarray]:
    data = pd.read_csv(name)
    x = data.loc[:, ((data.columns != 'cena') & (data.columns != 'dzielnica'))]
    dum = data['dzielnica']
    dum = pd.get_dummies(dum)
    x = pd.concat([x, dum], axis=1)
    y = data['cena']
    return x.to_numpy().astype(float), y.to_numpy().astype(float)

xs, ys = load2('mieszkania.csv')
xs, ys

(array([[104.,   2.,   2., ...,   1.,   0.,   0.],
        [ 43.,   1.,   1., ...,   0.,   1.,   0.],
        [128.,   3.,   2., ...,   0.,   0.,   0.],
        ...,
        [107.,   2.,   2., ...,   0.,   0.,   0.],
        [117.,   3.,   2., ...,   0.,   0.,   0.],
        [ 56.,   2.,   1., ...,   0.,   0.,   1.]]),
 array([ 780094.,  346912.,  523466.,  830965., 1090479.,  599060.,
         463639.,  166785.,  318849., 1011395.,  429462., 1051608.,
         563473.,  620449., 1097777.,  645200.,  776122.,  367862.,
         257190., 1010002.,  381594.,  980560.,  291748.,  788216.,
        1064711.,  758000.,  652028.,  515061.,  505024.,  479861.,
         550903.,  112635.,  291166.,  128308.,  629152.,  482861.,
         330775.,  453511.,  232030.,  392325.,  306354.,  922621.,
         452189.,  994035.,  488394.,  384485.,  105554.,  496032.,
         528212.,  156820.,  390186.,  331698.,  135225.,  712851.,
         402870.,  637259.,  660215.,  434946.,  252762., 1050412.,

In [23]:
def predict(weights, bias, xs):
    return xs @ weights + bias

def evaluate(weights, bias, xs, ys):
    ps = predict(weights, bias, xs)
    return msle(ys, ps)

In [24]:
##########################################################
# TODO: Implement linear regression and compute its msle #
#########################################################
def train(xs=xs, ys=ys):
  b, f = xs.shape
  weights = np.random.uniform(low=-1 / np.sqrt(f), high=1 / np.sqrt(f), size=[f]) # our initial guess for _a
  bias =  np.mean(ys) # our initial guess for _b
  lr = 1e-7 # step size
  n_epochs = 100000 # number of passes over the training data

  for i in range(n_epochs):
    prediction = predict(weights, bias, xs)
    grad_weights = 2/b * (xs.T @ (prediction - ys))
    grad_bias = 2 * np.mean(prediction - ys)
    weights -= lr * grad_weights
    bias -= lr * grad_bias
  return weights, bias

x_test, y_test = load2('mieszkania_test.csv')

weights1, bias1 = train()
evaluate(weights1, bias1, x_test, y_test)


0.07848135196344946

Note that the loss function that the algorithms optimizes (i.e $MSE$) differs from $MSLE$. We've already seen that this may result in a suboptimal solution.

How can you change the setting so that we optimze $MSLE$ instead?

Hint:
<sub><sup><sub><sup><sub><sup>
Be lazy. We don't want to change the algorithm.
</sup></sub></sup></sub></sup></sub>

In [25]:
#############################################
# TODO: Optimize msle and compare the error #
#############################################
def train1(ys=ys, xs=xs):
  b, f = xs.shape
  weights = np.random.uniform(low=-1 / np.sqrt(f), high=1 / np.sqrt(f), size=[f]) # our initial guess for _a
  bias =  magic_constant # our initial guess for _b
  lr = 1e+4 # step size
  n_epochs = 400000 # number of passes over the training data
  for i in range(n_epochs):
    prediction = predict(weights, bias, xs)
    grad_help = 2 * (np.log(prediction + 1) - np.log(ys + 1)) / (1 + prediction)
    grad_bias = np.mean(grad_help)
    grad_weights = (1 / b) * xs.T @ grad_help
    weights -= lr * grad_weights
    bias -= lr * grad_bias
  return weights, bias
weights, bias = train1()
evaluate(weights, bias, x_test, y_test)

0.07861384941158826

Without any feature engineering our model approximates the price as a linear combination of original features:
$$
\text{price} \approx w_1 \cdot \text{area} + w_2 \cdot \text{district} + \dots.
$$
Let's now introduce some interactions between the variables. For instance, let's consider a following formula:
$$
\text{price} \approx w_1 \cdot \text{area} \cdot \text{avg. price in the district per sq. meter} + w_2 \cdot \dots + \dots.
$$
Here, we model the price with far greater granularity, and we may expect to see more acurate results.

Add some feature engineering to your model. Be sure to play with the data and not with the algorithm's code.

Think how to make sure that your model is capable of capturing the $w_1 \cdot \text{area} \cdot \text{avg. price...}$ part, without actually computing the averages.

Hint:
<sub><sup><sub><sup><sub><sup>
Is having a binary encoding for each district and multiplying it by area enough?
</sup></sub></sup></sub></sup></sub>

Hint 2:
<sub><sup><sub><sup><sub><sup>
Why not multiply everything together? I.e. (A,B,C) -> (AB,AC,BC).
</sup></sub></sup></sub></sup></sub>

In [26]:
def load3(name: str) -> Tuple[np.ndarray, np.array, np.ndarray]:
    data = pd.read_csv(name)
    data['cenazametr'] = data['cena']/(data['m2'])
    x = data.loc[:, ((data.columns != 'cena') & (data.columns != 'dzielnica'))]
    dum = data['dzielnica']
    dum = pd.get_dummies(dum)
    x = pd.concat([x, dum], axis=1)
    y = data['cena'].to_numpy()
    return x, y

In [27]:
###############################################
# TODO: Implement the feature engineering part#
###############################################
xs, ys = load3('mieszkania.csv')
xs


,m2,ilość_sypialni,ilość_łazienek,rok_budowy,parking_podziemny,cenazametr,grodziskowo,mokotowo,ochotowo,wolowo
0,104,2,2,1940,1,7500.903846,False,True,False,False
1,43,1,1,1970,1,8067.720930,False,False,True,False
2,128,3,2,1916,1,4089.578125,True,False,False,False
3,112,3,2,1920,1,7419.330357,False,True,False,False
4,149,3,3,1977,0,7318.651007,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...
195,54,1,1,1992,1,7015.370370,False,True,False,False
196,35,1,1,1985,1,4852.342857,True,False,False,False
197,107,2,2,1935,0,4422.252336,True,False,False,False
198,117,3,2,1978,1,3684.410256,True,False,False,False


In [28]:
##############################################################
# TODO: Test your solution on the training and test datasets #
##############################################################